# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(f"Current working directory is : {os.getcwd()}")

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

Current working directory is : /home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:2])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91'], ['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73']]


In [4]:
# checking the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This is making a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# For establish connection and start execution of queries, creating a session
session = cluster.connect()

print("Established Connection")

Established Connection


#### Create Keyspace

In [6]:
#create a Keyspace using the execute() 

session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify 
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
##Query_1: getting the artist, song title and song's length in the music app history that was heard during \
##sessionId = 338, and itemInSession = 4

#Creating table for query_1 
TO_DO_query_1 = """CREATE TABLE IF NOT EXISTS session_item (sessionId int,itemInSession int,artist text,song text, length float, PRIMARY KEY (sessionId, itemInSession))"""

session.execute(TO_DO_query_1)

print("Table Created...")

Table Created...


In [9]:
# here are setting up the CSV file. and inserting data
#use data  file
file = 'event_datafile_new.csv'

#reading CSV files and inserting table rows into Apache Cassandra
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip headers
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variables
        query = "INSERT INTO session_item ( sessionId, itemInSession, artist, song, length) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        ## Assign columns element should be assigned for each one column in the INSERT statement.
        ## e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        
        session.execute(query, (int(line[8]), int(line[3]), line[0],line[10], float(line[5])) )
    print("Inserted rows into Cassandra Table")

Inserted rows into Cassandra Table


#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## SELECT statement for checking the data was entered into the table
select_1 = "select artist, song, length from session_item WHERE sessionId = 338 and itemInSession = 4"
#Executin of  query
rows = session.execute(select_1)

#show data
for row in rows:
    print (row)

Row(artist='Faithless', song='50', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

Creating Query_2 :

For query_2, we need a way to run query on sessionId and userId. Also, we need the data sorted on itemInSession. So, our primary key must have these columns. We can partition the data on a composite key (sessionId, userId).

Select query : select artist,song,itemInSession,song, firstName, lastName from user_session\
            WHERE userId = 10 and sessionId = 182

Our Primary key will be ((sessionId, userId), itemInSession)), where (sessionId, userId) is the partition key and itemInSession is the clustering column.

Also, we are using the clause - WITH CLUSTERING ORDER BY (itemInSession ASC), to sort our data based on itemInSession Columns we included in the table : sessionId, userId, itemInSession, artist, song, firstName, lastName

In [11]:
## Query_2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

create_query_2 = "CREATE TABLE IF NOT EXISTS user_session"

create_query_2 = create_query_2 + "(sessionId int, userId int, itemInSession int, artist text, song text, firstName text, lastName text,  PRIMARY KEY ((sessionId, userId), itemInSession)) WITH CLUSTERING ORDER BY (itemInSession ASC)"

session.execute(create_query_2)

print('Creating Tables as per create_query_2')

Creating Tables as per create_query_2


In [12]:
#use even CSV file
file = 'event_datafile_new.csv'

#reading CSV files and inserting table rows into Apache Cassandra
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip headers
    for line in csvreader:
        
        ## Assign the INSERT statements into the `query` variables
        query = "INSERT INTO user_session (sessionId, userId, itemInSession , artist, song, firstName, lastName) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s) "
        
        # Assign columns element should be assigned for each one column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), line[0], line[9], line[1], line[4]))
        
    print("Inserted data rows into Table")

Inserted data rows into Table


In [13]:
## SELECT statement for checking the data was entered into the table

select_query_2 = "select artist,song,itemInSession,song, firstName, lastName from user_session\
            WHERE userId = 10 and sessionId = 182"
#Execution of query
rows = session.execute(select_query_2)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession=0, song_="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession=1, song_='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession=2, song_='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3, song_='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


Creating Query_3:

For query_3, we need a way to run query on song. So, our primary key must have song. Also, the query should be such that it does not contain duplicate users for a song. So we need to model data in such a way that we don't allow duplicate users for a song in our table. This can be acheived by including userId in our primary key.

Select query : SELECT song, firstName, lastName FROM user_song where song = 'All Hands Against His Own'

Our Primary key will be ((song), userId)), where song is the partition key and userId is the clustering column. Columns we included in the table : song, userId, firstName, lastName

In [14]:
## Query_3: Finding every user name (first and last) in music app history whoever listened to the song 'All Hands Against His Own'
#creating table as per query_3

create_query_3 = "CREATE TABLE IF NOT EXISTS song"
create_query_3 = create_query_3 + "(song text, userId text, firstName text, lastName text,\
                    PRIMARY KEY (song, userId))"

#Execution of query
session.execute(create_query_3)
print('Table created as per create_query_3')

Table created as per create_query_3


In [15]:
#use even CSV file
file = 'event_datafile_new.csv'

#reading CSV files and inserting table rows into Apache Cassandra
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip headers
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variables
        query = "INSERT INTO song (song,userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[10], line[1],line[4]))

    print("Inserted data rows into Table")

Inserted data rows into Table


In [16]:
## SELECT statement for checking the data was entered into the table
select_query_3 = "select song, firstName, lastName from song WHERE song='All Hands Against His Own'"

#Execution of query
rows = session.execute(select_query_3)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [17]:
## Execution has done so Droping the table before closing out the sessions
drop_query_1 = "DROP TABLE IF EXISTS sparkify.session_item"
rows = session.execute(drop_query_1)
    
drop_query_2 = "DROP TABLE IF EXISTS sparkify.user_session"
rows = session.execute(drop_query_2)
    
drop_query_3 = "DROP TABLE IF EXISTS sparkify.user_song"
rows = session.execute(drop_query_3)
print('Execution done for all drop_query')

Execution done for all drop_query


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()